In [1]:
pip install noisereduce


In [2]:
pip install praat-parselmouth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 26.1 MB/s eta 0:00:00


In [3]:
!pip install --upgrade librosa

In [4]:
pip install librosa scipy

In [5]:
pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00


In [6]:
pip install pysptk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.9/461.9 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pysptk: filename=pysptk-1.0.1-cp310-cp310-linux_x86_64.whl size=1241915 sha256=f451ce4a18291cb54ff4850f83cf7b0ef86f502b4b978410b2c563e38674f730
  Stored in directory: /root/.cache/pip/wheels/ba/c2/7f/73c54e1795eeca8f995f95474c8290b78ee0a6306412dc934f
Successfully built pysptk


In [27]:
import pandas as pd
import numpy as np
import librosa
import parselmouth
from parselmouth.praat import call
import scipy.stats as stats
import scipy
from scipy.signal.windows import hamming
import pysptk
from scipy.io import wavfile
from tqdm import tqdm
import librosa
import numpy as np
import scipy
import pysptk
from scipy.io import wavfile
import noisereduce as nr
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import os

import librosa
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import os
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Dropout, Attention
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import (
    accuracy_score,  # For calculating accuracy
    roc_auc_score,   # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)
import scipy
from scipy.io import wavfile
from tqdm import tqdm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load File

In [16]:
#df = pd.read_csv("IEMOCAP_Updated.csv")
df = pd.read_csv('/content/drive/Shared drives/CS298/CS 297/IEMOCAP//IEMOCAP_Updated.csv')
df['Audio_Uttrance_Path'] = df['Audio_Uttrance_Path'].str.replace('/content/drive/My Drive', '/content/drive/Shareddrives/CS298')
merged_df = df[['Speaker_id', 'Audio_Uttrance_Path', 'Major']].rename(columns={'Major': 'Emotion'})
emotion_mapping = {
    'Neutral state': 'neutral',
    'Frustration': 'sad',
    'Anger': 'angry',
    'Surprise': None,  # Assuming 'Surprise' to 'neutral' if not sure where to map
    'Disgust': None,  # Mapping 'Disgust' to 'angry' or consider removing if not relevant
    'Other': None,  # Remove 'Other' or map if it can be categorized
    'Sadness': 'sad',
    'Fear': None,  # Assuming 'Fear' to 'neutral' or map to 'angry' based on context
    'Happiness': 'happy',
    'Excited': 'happy'
}
# Apply the mapping to the Emotion column
merged_df['Emotion'] = merged_df['Emotion'].map(emotion_mapping)

# Remove any rows with None as Emotion (unmapped categories)
merged_df = merged_df.dropna(subset=['Emotion'])
merged_df
merged_df['Emotion'] = merged_df['Emotion'].str.strip().str.lower()
merged_df.reset_index(drop=True, inplace=True)
merged_df.head()

,Speaker_id,Audio_Uttrance_Path,Emotion
0,Ses04F_script03_1_M003,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,happy
1,Ses04F_script03_1_F003,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,happy
2,Ses04F_script03_1_M004,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,happy
3,Ses04F_script03_1_F004,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,happy
4,Ses04F_script03_1_F005,/content/drive/Shareddrives/CS298/CS 297/IEMOC...,happy


In [9]:
len(merged_df)

9511

In [17]:
merged_df['Audio_Uttrance_Path'][0]

'/content/drive/Shareddrives/CS298/CS 297/IEMOCAP/Session4/Audio_Uttrances/Ses04F_script03_1/Ses04F_script03_1_M003.wav'

## Test the audio Sample

In [11]:
def extract_extended_features(audio_path):
    # Load the audio file
    y, sr = librosa.load(audio_path, sr=16000, mono=True)

    # Reduce noise using noisereduce
    # y = nr.reduce_noise(y=y, sr=sr, thresh_n_mult_nonstationary=2, stationary=False)

    # Features dictionary
    features = {}

    # MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=45)
    for i, mfcc in enumerate(mfccs):
        features[f'MFCC_{i}'] = np.mean(mfcc)

    # Pitch
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_mean = np.mean(pitches[pitches > 0])
    pitch_std = np.std(pitches[pitches > 0])
    features['Pitch_mean'] = pitch_mean
    features['Pitch_std'] = pitch_std

    # Energy
    rmse = librosa.feature.rms(y=y)
    features['Energy_mean'] = np.mean(rmse)
    features['Energy_std'] = np.std(rmse)

    # Spectral Centroid
    spec_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)
    features['Spectral_Centroid_mean'] = np.mean(spec_centroids)
    features['Spectral_Centroid_std'] = np.std(spec_centroids)

    # Spectral Bandwidth, Rolloff, RMS Energy
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features['Spectral_Bandwidth_Mean'] = np.mean(spectral_bandwidth)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    features['Spectral_Rolloff_Mean'] = np.mean(rolloff)
    rms_energy = librosa.feature.rms(y=y)
    features['RMS_Energy_Mean'] = np.mean(rms_energy)

    # Chroma Features
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    for i, chroma in enumerate(chroma_stft):
        features[f'Chroma{i}'] = np.mean(chroma)

    # Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y)
    features['ZCR_mean'] = np.mean(zcr)
    features['ZCR_std'] = np.std(zcr)

    # Loudness (using Energy)
    S, phase = librosa.magphase(librosa.stft(y))
    rms_s = librosa.feature.rms(S=S)
    features['Loudness'] = np.mean(librosa.amplitude_to_db(rms_s, ref=np.max))

    # Mel-Spectrogram
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
    features['Mel_Spec_Mean'] = np.mean(mel_spec)

    # LPC - Linear Predictive Coefficients
    lpc_coefficients = librosa.lpc(y, order=10)
    for i, coeff in enumerate(lpc_coefficients):
        features[f'LPC_{i}'] = coeff

    return features


## Feature Extraction

In [19]:
features_list = []

for index, row in tqdm(merged_df.iterrows(), total=len(merged_df), desc="Extracting features"):
    features = extract_extended_features(row['Audio_Uttrance_Path'])
    features['Emotion'] = row['Emotion']
    features['Speaker_id'] = row['Speaker_id']
    features_list.append(features)


Extracting features: 100%|██████████| 9511/9511 [1:48:38<00:00,  1.46it/s]


In [31]:
df = pd.DataFrame(features_list)
df.to_csv("features_labeled_majorColumn.csv", index=False)

In [33]:
features_df = pd.read_csv("features_labeled_majorColumn.csv")

In [24]:
features_df.head()

,MFCC_0,MFCC_1,MFCC_2,MFCC_3,MFCC_4,MFCC_5,MFCC_6,MFCC_7,MFCC_8,MFCC_9,...,LPC_3,LPC_4,LPC_5,LPC_6,LPC_7,LPC_8,LPC_9,LPC_10,Emotion,Speaker_id
0,-497.43484,140.760960,-16.455850,41.658283,-7.045522,7.846185,-6.505238,4.318293,-8.105709,7.961764,...,-2.485955,2.729523,-2.626336,2.099214,-1.537163,1.199934,-0.744857,0.237278,happy,Ses04F_script03_1_M003
1,-443.28308,106.770775,-15.674707,41.263980,-16.081266,19.867590,-13.291323,3.684966,-14.477111,5.517625,...,-1.919262,1.902377,-1.845863,1.762506,-1.495473,1.111268,-0.653970,0.194584,happy,Ses04F_script03_1_F003
2,-435.87375,120.006454,-24.540840,46.357655,-5.625869,-1.839700,-7.100257,-1.048477,-5.745254,10.270786,...,-4.136401,3.242918,-2.986958,3.273842,-2.974241,1.904652,-0.769274,0.175338,happy,Ses04F_script03_1_M004
3,-328.74980,97.679440,-40.115190,26.156448,-20.433031,8.588634,-28.492516,-6.059148,-3.679716,2.988311,...,-4.499185,4.571260,-4.274839,3.875180,-3.189097,2.188988,-1.171403,0.389666,happy,Ses04F_script03_1_F004
4,-352.65040,68.832466,-20.212467,16.935734,-22.049334,8.931537,-15.522003,-6.032151,-2.046423,5.841474,...,-3.112117,3.180664,-2.962393,2.690530,-2.234719,1.561520,-0.864859,0.355253,happy,Ses04F_script03_1_F005


In [25]:
emotion_counts = features_df['Emotion'].str.strip().str.lower().value_counts()

# Print unique values along with their counts
for emotion, count in emotion_counts.items():
    print(f"{emotion}: {count}")

sad: 3650
happy: 2306
neutral: 2277
angry: 1278


In [28]:

X = features_df.iloc[:, :81]
y = features_df['Emotion']
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
#X_scaled = X
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [29]:

def build_model(X_train_reshaped):
    inputs = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
    x = Bidirectional(LSTM(units=128, return_sequences=True))(inputs)  # Replace hp.Int with fixed value
    #x = Dropout(rate=0.25)(x)  # Replace hp.Float with fixed value
    # Attention layer
    query_value_attention_seq = Attention()([x, x])
    x = LSTM(64)(query_value_attention_seq)
    outputs = Dense(y_categorical.shape[1], activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [30]:
# Assuming X_scaled and y_categorical are already defined and preprocessed

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Arrays to store scores for each fold
fold_accuracies = []
fold_f1_scores = []
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
fold = 1
# Loop through each fold
for train_index, test_index in kf.split(X_scaled):
    print('Fold ', fold)
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y_categorical[train_index], y_categorical[test_index]

    # Reshape input for LSTM layers
    X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    '''
    # Define the model
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
    #model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.2))
    model.add(Dense(y_categorical.shape[1], activation='softmax'))
    '''
    model = build_model(X_train_reshaped)
    # Compile the model
    optimizer = Adam(learning_rate=0.00001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train_reshaped, y_train, epochs=50, batch_size=30, validation_split=0.2, verbose=2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test_reshaped, y_test, verbose=0)
    print(f'Fold Test Accuracy: {accuracy * 100:.2f}%')
    fold_accuracies.append(accuracy)

    # Predict and calculate F1 score
    y_pred_probs = model.predict(X_test_reshaped)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)
    f1 = f1_score(y_true, y_pred, average='macro')
    print(f'F1 Score: {f1:.4f}')
    fold_f1_scores.append(f1)
    class_report = classification_report(y_true, y_pred)
    print("Classification Report:\n", class_report)
    fold+=1
    print('----------------------------------------------')


# Calculate average accuracy and F1 score across all folds
avg_accuracy = np.mean(fold_accuracies)
avg_f1_score = np.mean(fold_f1_scores)

print(f'Average Test Accuracy: {avg_accuracy * 100:.2f}%')
print(f'Average F1 Score: {avg_f1_score:.4f}')


Fold  1
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


203/203 - 10s - 50ms/step - accuracy: 0.3055 - loss: 1.3812 - val_accuracy: 0.3154 - val_loss: 1.3795
Epoch 2/50
203/203 - 2s - 9ms/step - accuracy: 0.3690 - loss: 1.3701 - val_accuracy: 0.3568 - val_loss: 1.3700
Epoch 3/50
203/203 - 3s - 14ms/step - accuracy: 0.3953 - loss: 1.3593 - val_accuracy: 0.3824 - val_loss: 1.3605
Epoch 4/50
203/203 - 4s - 19ms/step - accuracy: 0.4050 - loss: 1.3486 - val_accuracy: 0.3890 - val_loss: 1.3511
Epoch 5/50
203/203 - 3s - 17ms/step - accuracy: 0.4109 - loss: 1.3379 - val_accuracy: 0.4014 - val_loss: 1.3416
Epoch 6/50
203/203 - 3s - 12ms/step - accuracy: 0.4164 - loss: 1.3270 - val_accuracy: 0.4021 - val_loss: 1.3321
Epoch 7/50
203/203 - 2s - 12ms/step - accuracy: 0.4198 - loss: 1.3163 - val_accuracy: 0.4060 - val_loss: 1.3227
Epoch 8/50
203/203 - 3s - 16ms/step - accuracy: 0.4247 - loss: 1.3060 - val_accuracy: 0.4093 - val_loss: 1.3135
Epoch 9/50
203/203 - 3s - 16ms/step - accuracy: 0.4277 - loss: 1.2963 - val_accuracy: 0.4120 - val_loss: 1.3053
Epo

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
F1 Score: 0.4013
Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.28      0.35       283
           1       0.50      0.37      0.43       441
           2       0.44      0.19      0.26       495
           3       0.44      0.78      0.56       684

    accuracy                           0.46      1903
   macro avg       0.47      0.40      0.40      1903
weighted avg       0.46      0.46      0.42      1903

----------------------------------------------
Fold  2
Epoch 1/50


...

60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
F1 Score: 0.4187
Classification Report:
               precision    recall  f1-score   support

           0       0.45      0.34      0.39       254
           1       0.46      0.35      0.40       477
           2       0.45      0.23      0.30       432
           3       0.48      0.74      0.59       739

    accuracy                           0.47      1902
   macro avg       0.46      0.42      0.42      1902
weighted avg       0.47      0.47      0.45      1902

----------------------------------------------
Fold  5
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


203/203 - 8s - 38ms/step - accuracy: 0.2998 - loss: 1.3770 - val_accuracy: 0.3620 - val_loss: 1.3695
Epoch 2/50
203/203 - 3s - 17ms/step - accuracy: 0.3609 - loss: 1.3661 - val_accuracy: 0.3817 - val_loss: 1.3602
Epoch 3/50
203/203 - 2s - 11ms/step - accuracy: 0.3977 - loss: 1.3552 - val_accuracy: 0.3968 - val_loss: 1.3509
Epoch 4/50
203/203 - 2s - 11ms/step - accuracy: 0.4104 - loss: 1.3442 - val_accuracy: 0.4014 - val_loss: 1.3413
Epoch 5/50
203/203 - 2s - 8ms/step - accuracy: 0.4173 - loss: 1.3329 - val_accuracy: 0.4060 - val_loss: 1.3317
Epoch 6/50
203/203 - 3s - 13ms/step - accuracy: 0.4230 - loss: 1.3216 - val_accuracy: 0.4100 - val_loss: 1.3222
Epoch 7/50
203/203 - 2s - 9ms/step - accuracy: 0.4293 - loss: 1.3105 - val_accuracy: 0.4074 - val_loss: 1.3130
Epoch 8/50
203/203 - 4s - 19ms/step - accuracy: 0.4317 - loss: 1.3000 - val_accuracy: 0.4060 - val_loss: 1.3045
Epoch 9/50
203/203 - 4s - 18ms/step - accuracy: 0.4347 - loss: 1.2904 - val_accuracy: 0.4047 - val_loss: 1.2971
Epoch

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
F1 Score: 0.4060
Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.27      0.33       247
           1       0.42      0.38      0.40       466
           2       0.48      0.23      0.31       445
           3       0.48      0.73      0.58       744

    accuracy                           0.47      1902
   macro avg       0.46      0.40      0.41      1902
weighted avg       0.46      0.47      0.44      1902

----------------------------------------------
Average Test Accuracy: 46.98%
Average F1 Score: 0.4099
